<a href="https://colab.research.google.com/github/2303a51745/RL-b-11/blob/main/Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
# Monte Carlo Policy Evaluation & Control (Colab)
# ===============================================

import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt

# -------------------------
# GridWorld Environment
# -------------------------
class GridWorld:
    ACTIONS = [0, 1, 2, 3]   # 0:UP, 1:RIGHT, 2:DOWN, 3:LEFT
    DIRS = {0:(-1,0), 1:(0,1), 2:(1,0), 3:(0,-1)}
    ARROWS = {0:'↑', 1:'→', 2:'↓', 3:'←'}

    def __init__(self, H=4, W=4, walls=None, terminals=None, step_cost=-0.04):
        self.H, self.W = H, W
        self.walls = set(walls or [])
        self.terminals = dict(terminals or {})
        self.step_cost = step_cost
        self.states = [(r,c) for r in range(H) for c in range(W) if (r,c) not in self.walls]

    def in_bounds(self, r, c):
        return 0 <= r < self.H and 0 <= c < self.W

    def is_terminal(self, s):
        return s in self.terminals

    def step(self, s, a):
        """Return next_state, reward, done"""
        if self.is_terminal(s):
            return s, 0, True
        r, c = s
        dr, dc = self.DIRS[a]
        nr, nc = r + dr, c + dc
        if (not self.in_bounds(nr,nc)) or ((nr,nc) in self.walls):
            ns = (r,c)
        else:
            ns = (nr,nc)
        reward = self.step_cost
        if ns in self.terminals:
            reward += self.terminals[ns]
        return ns, reward, self.is_terminal(ns)

    def reset(self):
        # Start in a random non-terminal, non-wall state
        return random.choice([s for s in self.states if not self.is_terminal(s)])


# -------------------------
# Monte Carlo: Policy Evaluation
# -------------------------
def mc_policy_evaluation(env, policy, episodes=5000, gamma=0.99):
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    V = defaultdict(float)

    for _ in range(episodes):
        # Generate an episode
        episode = []
        s = env.reset()
        while True:
            a = policy[s]
            ns, r, done = env.step(s, a)
            episode.append((s,a,r))
            s = ns
            if done: break

        # First-visit MC
        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            s, a, r = episode[t]
            G = gamma * G + r
            if s not in visited:
                returns_sum[s] += G
                returns_count[s] += 1
                V[s] = returns_sum[s] / returns_count[s]
                visited.add(s)
    return V


# -------------------------
# Monte Carlo Control (ε-greedy)
# -------------------------
def mc_control_epsilon_greedy(env, episodes=10000, gamma=0.99, epsilon=0.1):
    Q = defaultdict(lambda: np.zeros(len(env.ACTIONS)))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)

    def policy_fn(state):
        if random.random() < epsilon:
            return random.choice(env.ACTIONS)
        else:
            return np.argmax(Q[state])

    for _ in range(episodes):
        # Generate an episode
        episode = []
        s = env.reset()
        while True:
            a = policy_fn(s)
            ns, r, done = env.step(s, a)
            episode.append((s,a,r))
            s = ns
            if done: break

        # Compute returns
        G = 0
        visited = set()
        for t in reversed(range(len(episode))):
            s, a, r = episode[t]
            G = gamma * G + r
            if (s,a) not in visited:
                returns_sum[(s,a)] += G
                returns_count[(s,a)] += 1
                Q[s][a] = returns_sum[(s,a)] / returns_count[(s,a)]
                visited.add((s,a))

    # Derive greedy policy
    policy = {}
    for s in env.states:
        if env.is_terminal(s):
            policy[s] = None
        else:
            policy[s] = np.argmax(Q[s])
    return Q, policy


# -------------------------
# Visualization Helpers
# -------------------------
def print_policy(env, policy, title="Policy"):
    print(title)
    for r in range(env.H):
        row = []
        for c in range(env.W):
            s = (r,c)
            if s in env.walls:
                row.append("■")
            elif s in env.terminals:
                row.append(f"T({env.terminals[s]:.1f})")
            elif policy[s] is None:
                row.append("•")
            else:
                row.append(GridWorld.ARROWS[policy[s]])
        print(" ".join(f"{x:>6}" for x in row))

def plot_values(env, V, title="State-Value Function"):
    A = np.zeros((env.H, env.W))
    A[:] = np.nan
    for s in env.states:
        A[s] = V[s]
    plt.figure(figsize=(5,5))
    im = plt.imshow(A, cmap="coolwarm", interpolation='nearest')
    plt.title(title)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    for r in range(env.H):
        for c in range(env.W):
            if (r,c) in env.walls:
                txt = '■'
            elif (r,c) in env.terminals:
                txt = f"T\n{env.terminals[(r,c)]:.1f}"
            else:
                txt = f"{A[r,c]:.2f}"
            plt.text(c, r, txt, ha='center', va='center', fontsize=9)
    plt.axis('off')
    plt.show()


# -------------------------
# Example Run
# -------------------------
walls = {(1,1)}
terminals = {(0,3): +1.0, (1,3): -1.0}
env = GridWorld(H=4, W=4, walls=walls, terminals=terminals, step_cost=-0.04)

# Random policy evaluation
random_policy = {s: np.random.choice(env.ACTIONS) if not env.is_terminal(s) else None for s in env.states}
V_mc = mc_policy_evaluation(env, random_policy, episodes=5000)
plot_values(env, V_mc, "MC Policy Evaluation (Random Policy)")

# Monte Carlo Control
Q_mc, policy_mc = mc_control_epsilon_greedy(env, episodes=10000, epsilon=0.1)
print_policy(env, policy_mc, "MC Control Policy")
